<a href="https://colab.research.google.com/github/kiranramnath007/ColabExperiments/blob/master/HICO_DET_Faster_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
NUM_CLASSES = 49

In [2]:
import torch, torchvision
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torchvision import transforms
import cv2
from PIL import Image
from torch import optim
from collections import OrderedDict
from torch.autograd import Variable
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import cv2
import time
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import average_precision_score
# For training
images = torch.rand(10, 3, 287, 640)

#7 box per image
boxes1 = torch.ones(5, 7, 4) #2 images, 7 bbox, 4 co-ords
labels1 = torch.randint(0,2, (5,NUM_CLASSES)).float()

#11 boxes per image
boxes2 = torch.zeros(5,11,4)
labels2 = torch.randint(0,2,(5,NUM_CLASSES)).float()

boxes = torch.unbind(boxes1) + torch.unbind(boxes2)
#boxes = torch.zeros(5,11,4)
labels = torch.cat((labels1, labels2), dim = 0)
lengths = [7,7,7,7,7,11,11,11,11,11,11]
#images = list(image for image in images)
targets = []
for i in range(len(images)):
    d = {}
    d['boxes'] = boxes[i]
    #d['labels'] = labels[i]
    targets.append(d)
#output = model(images, targets)

#Transform
transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()])

#Pack bounding box tensor into one 
from torch.nn.utils.rnn import pad_sequence
packed_boxes = pad_sequence(boxes, batch_first=True)
packed_boxes.shape

torch.Size([10, 11, 4])

In [0]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [4]:
import sys
sys.path.append("/content/drive/My Drive/Research/Code/")
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)
from fast_rcnn import *

Mounted at /content/drive/


In [5]:
# !cp /content/drive/My\ Drive/Research/Data/HICO_DET/subset.zip /content/subset.zip
# #!unzip /content/drive/My\ Drive/Research/Data/HICO_DET/subset.zip -d /content/drive/My\ Drive/Research/Data/images2/
# !unzip /content/subset.zip -d /content/images/

Archive:  /content/subset.zip
replace /content/images/subset/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [7]:
!cp -r /content/drive/My\ Drive/Research/Data/Tensors/ /content/
!mv /content/Tensors/ /content/Images/

mv: cannot move '/content/Tensors/' to '/content/Images/Tensors': Directory not empty


In [0]:
gpu_flag = False
if torch.cuda.is_available():
    gpu_flag = True
model = fastrcnn_resnet50_fpn(pretrained_backbone=True, num_classes = NUM_CLASSES, trainable=False,gpu = gpu_flag)

In [9]:
time_start = time.time()
model.train()
pos_weight = 10*torch.ones(NUM_CLASSES)
if gpu_flag:
    images = images.cuda()
    labels = labels.cuda()
    pos_weight = pos_weight.cuda()
    for t in targets:
        t["boxes"] = t["boxes"].cuda()
predictions2 = model(images, targets, labels, pos_weight = pos_weight, batch_size = 10, gpu = gpu_flag)
print(time.time()-time_start)

0.57574462890625


In [0]:
# For inference
model.eval()
pos_weight = 10*torch.ones(NUM_CLASSES)
predictions = model(images, targets, labels, pos_weight = pos_weight, batch_size = 10, gpu = gpu_flag)
predictions

In [0]:
# Set the maximum sequence length. 

class BucketDataset:

    def __init__(self, inputs, targets, labels, lengths, batch_size):
        self.inputs = inputs      # shape = (N, max_seq_len)
        self.targets = targets    # shape = (N, ) or None (e.g., for autoencoder I can simply use inputs)
        self.lengths = lengths    # shape = (N, )
        self.labels = labels
        self.lengths_tensor = torch.tensor(self.lengths)
        self.batch_size = batch_size
        self.current = -1
        self._generate_batch_map()

    def _generate_batch_map(self, equal_length=False):
        batch_map = OrderedDict()
        # Organize lengths, e.g., batch_map[10] = [30, 124, 203, ...] <= indices of sequences of length 10
        for idx, length in enumerate(self.lengths):
            if length not in batch_map:
                batch_map[length] = [idx]
            else:
                batch_map[length].append(idx)
        # Use batch_map to split indices into batches of equal size
        # e.g., for batch_size=3, batch_list = [[23,45,47], [49,50,62], [63,65,66], ...]
        self.batch_list = []
        for length, indices in batch_map.items():
            for group in [indices[i:(i+self.batch_size)] for i in range(0, len(indices), self.batch_size)]:
                self.batch_list.append((length,group))
        self.batch_list = sorted(self.batch_list)
        self.batch_list = [b[1] for b in self.batch_list]

    def batch_count(self):
        return len(self.batch_list)

    def __len__(self):
        return len(self.lengths)

    def __iter__(self):
        return self

    def __next__(self):
        self.current = self.current + 1
        if self.current > len(self.batch_list)-1:
            self.current = -1
            raise StopIteration
        else:
            t = set(self.batch_list[self.current])
            indices = [True if index in t else False for index in range(len(self.inputs))]
            l = self.lengths_tensor[indices]

            return self.inputs[indices], \
                   [{"boxes":t} for t in self.targets[indices][:,:l[0],:]], \
                   self.labels[indices]

In [0]:
def step(bucket, test_bucket, optimizer, model, pos_weight, epoch, partition,scheduler, gpu = False):
    running_loss = 0.0
    num_buckets = len(bucket.batch_list)
    checkpoint_every = max(num_buckets // 2, 1)
    time_start = time.time()
    for index, mb in enumerate(bucket):
        time_start_getting_data = time.time()
        inputs, targets, labels = mb
        if gpu:
            inputs = inputs.cuda()
            for t in targets:
                t["boxes"] = t["boxes"].cuda()
            labels = labels.cuda()
            pos_weight = pos_weight.cuda()
        batch_size = len(targets)
        print(inputs.shape, labels.shape, targets[0]["boxes"].shape)
        # print("Time to get data from minibatch", time.time() - time_start_getting_data)
        
        time_start_forward = time.time()
        # zero the parameter gradients
        optimizer.zero_grad()
        model.train()
        # forward + backward + optimize
        #print("Labels ",labels.shape," image ",inputs.shape, " batch_size ", batch_size)
        losses = model(inputs, targets, labels, batch_size, pos_weight, gpu)
        loss = losses["loss_classifier"]
        
        # print("Time taken for forward pass", time.time() - time_start_forward)
        
        time_start_backward_prop = time.time()        
        loss.backward()
        optimizer.step()
        # print("time taken for backward propagation", time.time() - time_start_backward_prop)
        print("Batch size", batch_size, "Loss", loss.item())
        running_loss += loss.item()
        
        if index % checkpoint_every == 0:
            checkpoint = {'model': model,
                      'state_dict': model.state_dict(),
                      'optimizer' : optimizer.state_dict()}

            torch.save(checkpoint, 'action_detector_epoch_'+str(epoch)+'_partition_'+str(partition)+'.pth')
            model.eval()
            with torch.no_grad():
                y_pred_train = model(inputs, targets, labels, batch_size, pos_weight, gpu)
                if gpu:
                    y_pred_train = y_pred_train.cpu()
                    labels = labels.cpu()
                train_average_precision = average_precision_score(labels, y_pred_train, average="micro")
                print("Train average precision is ", train_average_precision)
                
                test_average_precision = 0
                if test_bucket is None:
                    continue
                for test_mb in test_bucket:
                    inputs, targets, labels = test_mb
                    batch_size = len(targets)
                    if gpu:
                        inputs, labels = inputs.cuda(), labels.cuda()
                        for t in targets:
                            t["boxes"] = t["boxes"].cuda()
                            
                    y_pred = model(inputs, targets, labels, batch_size, pos_weight, gpu)            
                    if gpu:
                        y_pred = y_pred.cpu()
                        labels = labels.cpu()
                    test_average_precision += average_precision_score(labels, y_pred, average="micro")
                print("Test average precision", test_average_precision / len(test_bucket))
        scheduler.step()        
        print("Epoch",epoch,"Iteration ",index,"Partition",partition,"running_loss",running_loss)
        # n_iter = epoch*2400 + partition*111 + iteration
        # writer.add_scalar('Loss/train', np.random.random(), n_iter)
        # writer.add_scalar('Loss/test', np.random.random(), n_iter)
        # writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
        # writer.add_scalar('Accuracy/test', np.random.random(), n_iter)
        #print("Time taken for one iteration",time.time() - time_start_getting_data)
    return running_loss    

In [0]:
def train_net(model, test_bucket = None, epochs = 20, batch_size = 10, gpu = False):
    
    optimizer = optim.SGD(model.parameters(), lr=1e-2, weight_decay=1e-3, momentum=0.9)
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 18000, gamma=0.1)
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-4, max_lr=1e-3, step_size_up = 2400)
    pos_weight = 10*torch.ones(NUM_CLASSES) #10 is the weight attached to misclassified positive sample

    writer = SummaryWriter() #
    for epoch in range(epochs):
        print("Epoch number ",epoch)
        for partition in range(11):
            images = torch.load(data_directory + "train_images_" + str(partition) + ".pt")
            boxes = torch.load(data_directory + "train_boxes_" + str(partition) + ".pt")
            labels = torch.load(data_directory + "train_labels_" + str(partition) + ".pt")
            with (open(data_directory + "train_lengths_" + str(partition) + ".txt", "rb")) as picklefile:
              lengths = pickle.load(picklefile)
            buckets = BucketDataset(images, boxes, labels, lengths, batch_size)
            step(buckets, test_bucket, optimizer, model,pos_weight,epoch, partition,scheduler,gpu)

In [0]:
import json
anno_directory = "/content/drive/My Drive/Research/Data/"
data_directory = "/content/Images/"
with open(anno_directory + 'HICO_DET/subset_anno_list_with_bg_class.json') as f:
    data = json.load(f)

# data_directory = "../../Data/HICO/data_symlinks/"
# with open(data_directory + 'hico_processed/subset_anno_list_with_bg_class.json') as f:
#     data = json.load(f)

In [12]:
len(data)

13000

### Extract ground truth bounding boxes

In [0]:
#This gets one-image one-bbox data

def one_box_one_image_data(data):
    per_image_data = {}
    c = 0

    for image_data in data:
        global_id = image_data['global_id']
        for hoi in image_data['hois']:
            bbox = hoi['human_bboxes']
            label = hoi['id']
            image_path = image_data['image_path_postfix']
            image_size = image_data['image_size']
            per_image_data[c] = {"bbox":bbox, "label":int(label), "path":image_path,'image_size':image_size}
            c += 1

    l = []
    for key in per_image_data:
        l.append(per_image_data[key]["label"])

    lt = torch.tensor(l)
    one_hot = torch.nn.functional.one_hot(lt)

    for index,t in enumerate(one_hot):
        per_image_data[index]["label"] = t
    
    return per_image_data

In [0]:
#This gets one-image all-box data

def one_image_all_box_data(data):
    ret = []
    
    for image_data in data:
        global_id = image_data["global_id"]
        bboxes = []
        labels = []
        image_path = image_data['image_path_postfix']
        image_size = image_data["image_size"]
        
        for hoi in image_data["hois"]:
            bboxes += hoi["human_bboxes"]
            labels += [float(hoi["id"])]*len(hoi["human_bboxes"])
                    
        ret.append({"global_id":global_id, "boxes":bboxes, "labels":labels,
                            "image_path":image_path,"image_size":image_size})
    return ret
ret = one_image_all_box_data(data)            

In [12]:
len(ret)

13000

In [0]:
train_annot = [d for d in ret if d["image_path"].startswith("train")]
test_annot = [d for d in ret if d["image_path"].startswith("test")]

In [0]:
def resize_boxes(boxes, original_size, new_size):
    # type: (Tensor, List[int], List[int])
    ratios = [float(s) / float(s_orig) for s, s_orig in zip(new_size, original_size)]
    ratio_height, ratio_width = ratios
    xmin, ymin, xmax, ymax = boxes.unbind(1)

    xmin = xmin * ratio_width
    xmax = xmax * ratio_width
    ymin = ymin * ratio_height
    ymax = ymax * ratio_height
    return torch.stack((xmin, ymin, xmax, ymax), dim=1)

In [16]:
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import cv2
img_path = "../../Data/HICO/data_symlinks/hico_clean/images/"+ret[6953]["image_path"]
#img = Image.open(img_path) # Load the image
img = cv2.imread(img_path) # Read image with cv2
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert to RGB
coords= ret[6953]["boxes"]
for c in coords:
    cv2.rectangle(img, (int(c[0]), int(c[1])), (int(c[2]), int(c[3])),color=(0, 255, 0), thickness=1) # Draw Rectangle with the coordinates
#cv2.putText(img,pred_cls[i], boxes[i][0],  cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th) # Write the prediction class
plt.figure(figsize=(20,30)) # display the output image
plt.imshow(img)
plt.xticks([])
plt.yticks([])


plt.imshow(img)

error: ignored

In [0]:
# img = cv2.imread(img_path) # Read image with cv2
# img = cv2.resize(img, (640, 640), interpolation = cv2.INTER_LINEAR)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert to RGB
# coords= resize_boxes(torch.tensor(ret[30]["boxes"][0]),ret[30]["image_size"][:2],(640,640))
# coords = [int(t) for t in list(coords.squeeze(0))]
# cv2.rectangle(img, (int(coords[0]), int(coords[1])), (int(coords[2]), int(coords[3])),color=(0, 255, 0), thickness=3) # Draw Rectangle with the coordinates
# #cv2.putText(img,pred_cls[i], boxes[i][0],  cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th) # Write the prediction class
# plt.figure(figsize=(20,30)) # display the output image
# plt.imshow(img)
# plt.xticks([])
# plt.yticks([])

### Create train and test buckets

In [0]:
#Get number of boxes for each image
def create_tensors_for_data_loader(data):
    #lengths = [len(t["boxes"]) for t in train_subset]
    lengths = []
    #Create packed_boxes for all images together
    boxes = ()
    width = 640
    images = torch.empty(0,3,width,width)
    transform = transforms.Compose([transforms.Resize((width,width)),
                                    transforms.ToTensor()])
    labels_tensor = torch.empty(0,NUM_CLASSES)
    for t in data:
        size = t["image_size"]
        if len(t["boxes"]) == 0:
            image_boxes = torch.tensor([[0,0,size[1],size[0]]])
        else:
            image_boxes = torch.tensor(t["boxes"])    

        #img_path = data_directory + "hico_clean/images/subset/" + t["image_path"]
        img_path = data_directory + "hico_clean/images/subset/" + t["image_path"]
        img = Image.open(img_path)
        img = transform(img)
        img = img.unsqueeze(0)
        if img.shape[1] == 1:
            img = img.repeat(1,3,1,1)
        images = torch.cat((images,img), dim = 0)

        label_tensor = torch.zeros(1,NUM_CLASSES)
        labels = t["labels"]
        
        #Create one_hot encoding of image labels
        for label in labels:
            label_tensor[0,int(label)] = 1.

        labels_tensor = torch.cat((labels_tensor, label_tensor), dim = 0)
        resized_boxes = resize_boxes(image_boxes, size[:2], (width,width))
        boxes += torch.unbind(resized_boxes.unsqueeze(0))
        lengths.append(len(image_boxes))

    packed_boxes = pad_sequence(boxes, batch_first=True)
    return images, packed_boxes, labels_tensor, lengths

In [0]:
# data_directory = "/content/images/"
# import pickle, gc
# ts = time.time()
# batch_size = 10
# for i in range(11):
#   train_images, train_boxes, train_labels, train_lengths = create_tensors_for_data_loader(train_annot[i*1000:(i+1)*1000])
#   torch.save(train_images, data_directory + "train_images_"+str(i) +".pt")
#   torch.save(train_boxes, data_directory + "train_boxes_"+str(i) +".pt")
#   torch.save(train_labels, data_directory + "train_labels_"+str(i) +".pt")
  
#   with open(data_directory + "train_lengths_"+str(i) +".txt", "wb") as f:
#     pickle.dump(train_lengths, f)
  
#   gc.collect()
# print("Time taken", time.time() - ts)

In [0]:
test_images, test_boxes, test_labels, test_lengths = create_tensors_for_data_loader(test_annot)
# torch.save(train_images, data_directory + "test_images.pt")
# torch.save(train_boxes, data_directory + "test_boxes.pt")
# torch.save(train_labels, data_directory + "test_labels.pt")

# with open(data_directory + "test_lengths.txt", "wb") as f:
#   pickle.dump(train_lengths, f)

In [0]:
# from shutil import copyfile
# import os
# files = os.listdir("/content/images/")
# for file in files:
#   if "." in file:
#     copyfile("/content/images/"+file, "/content/drive/My Drive/Research/Data/Tensors/" + file)

In [0]:
import pickle
test_images = torch.load(data_directory + "test_images.pt")
test_boxes = torch.load(data_directory + "test_boxes.pt")
test_labels = torch.load(data_directory + "test_labels.pt")
with (open(data_directory + "test_lengths.txt", "rb")) as picklefile:
  test_lengths = pickle.load(picklefile)
test_bucket = BucketDataset(test_images, test_boxes, test_labels, test_lengths, batch_size = 10)

In [0]:
train_net(model, test_bucket, gpu = gpu_flag)

Epoch number  0
torch.Size([10, 3, 256, 256]) torch.Size([10, 49]) torch.Size([1, 4])
Batch size 10 Loss 0.8210586905479431
Train average precision is  0.017475585505123525
Test average precision 0.00440461355403573
Epoch 0 Iteration  0 Partition 0 running_loss 0.8210586905479431
torch.Size([10, 3, 256, 256]) torch.Size([10, 49]) torch.Size([1, 4])
Batch size 10 Loss 0.8071987628936768
Epoch 0 Iteration  1 Partition 0 running_loss 1.6282574534416199
torch.Size([10, 3, 256, 256]) torch.Size([10, 49]) torch.Size([1, 4])
Batch size 10 Loss 0.8207950592041016
Epoch 0 Iteration  2 Partition 0 running_loss 2.4490525126457214
torch.Size([10, 3, 256, 256]) torch.Size([10, 49]) torch.Size([1, 4])
Batch size 10 Loss 0.8205874562263489
Epoch 0 Iteration  3 Partition 0 running_loss 3.2696399688720703
torch.Size([10, 3, 256, 256]) torch.Size([10, 49]) torch.Size([1, 4])
Batch size 10 Loss 0.8221161961555481
Epoch 0 Iteration  4 Partition 0 running_loss 4.091756165027618
torch.Size([10, 3, 256, 256]